<a href="https://colab.research.google.com/github/Gqndhi/Coursera_Capstone/blob/master/CapstoneProject_Week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Capstone Project
-----

###1. Import Libraries

In [1]:
!pip install geopy
!pip install geocoder

In [2]:
import numpy as np 
import pandas as pd 
import json 
from geopy.geocoders import Nominatim 
import geocoder
import requests 
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print("Libraries imported.")

Libraries imported.


In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

###2. Scrapping the data from Wikipedia

In [4]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
extracting_data = requests.get(url).text
wiki_data = BeautifulSoup(extracting_data, 'lxml')
print("WikiPage Scraped Successfully")

WikiPage Scraped Successfully


In [5]:
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

content = wiki_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,
                              'Borough': borough,
                              'Neighborhood': neighborhood},ignore_index=True)

print('Data Collected Successfully')
toronto.head()

Data Collected Successfully


,Postalcode,Borough,Neighborhood
0,0,0,0
1,M1A\n,Not assigned\n,Not assigned
2,M2A\n,Not assigned\n,Not assigned
3,M3A\n,North York\n,Parkwoods
4,M4A\n,North York\n,Victoria Village


In [6]:
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
toronto.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [7]:
neighbors = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()

In [8]:
neighbors["Borough"]=neighbors.Borough.str.replace('\\n','')
neighbors["Postalcode"]=neighbors.Postalcode.str.replace('\\n','')
neighbors["Neighborhood"]=neighbors.Neighborhood.str.replace('\\n','')

neighbors.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [9]:
neighbors.drop(neighbors[neighbors['Borough']=='Not assigned'].index,axis=0,inplace=True)
neighbors.drop(neighbors[neighbors['Neighborhood']=='Not assigned'].index,axis=0,inplace=True)
neighbors.reset_index(drop=True,inplace=True)

neighbors.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


###3. Getting the coordinates

In [10]:
LatLng = pd.read_csv('Geospatial_Coordinates.csv')
LatLng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
neighbors['Latitude']=LatLng['Latitude']
neighbors['Longitude']=LatLng['Longitude']
neighbors.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


###4. Visualizing the data

In [12]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto, Canada {}, {}.'.format(latitude, longitude))

The coordinates of Toronto, Canada 43.6534817, -79.3839347.


In [13]:
map_To = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighbors['Latitude'], neighbors['Longitude'], neighbors['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_To)  
    
map_To

In [14]:
map_To.save('map_To.html')

###5. Using Foursquare API to explore the City

In [15]:
CLIENT_ID = 'KXZYZ1M4CQ2T3PYJTOWBNIHWAPQ5VO5VS3F2NAQSJECWDMVY' # your Foursquare ID
CLIENT_SECRET = 'KJKC3ZIACUVHZUISZ2KD13TTEPGGXBF3RMFF244B4MQUMVB5' # your Foursquare Secret
VERSION = '20200605' # Foursquare API version


In [16]:
df = neighbors
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000,LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, 
                             lng, v['venue']['name'], 
                             v['venue']['location']['lat'], v['venue']['location']['lng'], 
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [18]:
df = toronto_data
toronto_venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])
print("Done")

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(3871, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
2,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater
3,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
4,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery


In [23]:
toronto_venues.groupby(["Neighborhood"]).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",100,100,100,100,100,100
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 227 uniques categories.


In [28]:
"Shopping Mall" in toronto_venues['Venue Category'].unique()

True

###6. Analyze each neighborhood

In [32]:
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(3871, 228)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nudist Beach,Office,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [33]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped.head()

(39, 228)


,Neighborhoods,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,Hobby Shop,Hostel,Hotel,Hotel Bar,Hungarian Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nudist Beach,Office,Organic Grocery,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Racetrack,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio,Zoo
0,Berczy Park,0.00,0.02,0.0,0.00,0.01,0.02,0.00,0.01,0.00,0.00,0.0,0.0,0.01,0.0,0.01,0.02,0.01,0.01,0.02,0.00,0.00,0.01,0.00,0.00,0.0,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.06,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.01,0.00,0.10,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.01,0.00,0.0,0.0,0.0,0.01,0.01,0.01,0.0,0.02,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.01,0.0,0.0,0.03,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.01,0.01,0.01,0.00,0.0,0.0,0.02,0.0,0.00,0.0,0.01,0.00,0.0,0.01,0.00,0.0,0.0,0.01,0.00,0.02,0.00,0.00,0.00,0.0,0.01,0.0,0.00,0.06,0.0,0.0,0.01,0.0,0.00,0.0,0.0,0.02,0.03,0.00,0.00,0.00,0.01,0.00,0.0,0.02,0.00,0.01,0.00,0.0,0.00,0.01,0.0,0.0,0.01,0.00,0.01,0.00,0.0,0.00,0.00,0.02,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.04,0.

In [36]:
print(len(to_grouped[to_grouped["Shopping Mall"] > 0]))
to_mall = to_grouped[["Neighborhoods","Shopping Mall"]]
print(to_mall.shape)
to_mall.head()

16
(39, 2)


,Neighborhoods,Shopping Mall
0,Berczy Park,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.00
2,"Business reply mail Processing Centre, South C...",0.00
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00
4,Central Bay Street,0.01


###7. Cluster each neighborhood using KMeans

In [37]:
k = 3
to_cluster = to_mall.drop(["Neighborhoods"], 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(to_cluster)
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 0, 2, 1, 0, 0], dtype=int32)

In [40]:
to_clustered = to_mall.copy()
to_clustered["Cluster Labels"] = kmeans.labels_
to_clustered.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)

to_clustered.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Berczy Park,0.00,0
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0
2,"Business reply mail Processing Centre, South C...",0.00,0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0
4,Central Bay Street,0.01,1


In [ ]:
to_clustered = to_clustered.join(toronto_data.set_index("Neighborhood"), on="Neighborhood")
to_clustered.drop(["Postalcode","Borough"],axis=1,inplace=True)



In [46]:
print(to_clustered.shape)
to_clustered.head()

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.00,0,43.644771,-79.373306
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0,43.636847,-79.428191
2,"Business reply mail Processing Centre, South C...",0.00,0,43.662744,-79.321558
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0,43.628947,-79.394420
4,Central Bay Street,0.01,1,43.657952,-79.387383


In [47]:
print(to_clustered.shape)
to_clustered.sort_values(["Cluster Labels"], inplace=True)
to_clustered.head()

(39, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.0,0,43.644771,-79.373306
36,"The Danforth West, Riverdale",0.0,0,43.679557,-79.352188
35,The Beaches,0.0,0,43.676357,-79.293031
32,Studio District,0.0,0,43.659526,-79.340923
30,"St. James Town, Cabbagetown",0.0,0,43.667967,-79.367675


In [54]:
to_clustered['Cluster Labels'].value_counts()

0    23
1    13
2     3
Name: Cluster Labels, dtype: int64

###8. Visualizing the Clusters

In [51]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_clustered['Latitude'], to_clustered['Longitude'], to_clustered['Neighborhood'], to_clustered['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
map_clusters.save('map_ToClustered.html')

###9. Gaining Insights by examining the clusters

####Cluster 0


In [55]:
to_clustered.loc[to_clustered['Cluster Labels'] == 0]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Berczy Park,0.0,0,43.644771,-79.373306
36,"The Danforth West, Riverdale",0.0,0,43.679557,-79.352188
35,The Beaches,0.0,0,43.676357,-79.293031
32,Studio District,0.0,0,43.659526,-79.340923
30,"St. James Town, Cabbagetown",0.0,0,43.667967,-79.367675
27,Roselawn,0.0,0,43.711695,-79.416936
22,"Parkdale, Roncesvalles",0.0,0,43.648960,-79.456325
21,"North Toronto West, Lawrence Park",0.0,0,43.715383,-79.405678
20,"Moore Park, Summerhill East",0.0,0,43.689574,-79.383160
18,Lawrence Park,0.0,0,43.728020,-79.388790


####Cluster 1

In [57]:
to_clustered.loc[to_clustered['Cluster Labels'] == 1]



,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
34,"The Annex, North Midtown, Yorkville",0.01,1,43.672710,-79.405678
33,"Summerhill West, Rathnelly, South Hill, Forest...",0.01,1,43.686412,-79.400049
31,Stn A PO Boxes,0.01,1,43.646435,-79.374846
4,Central Bay Street,0.01,1,43.657952,-79.387383
29,St. James Town,0.01,1,43.651494,-79.375418
26,Rosedale,0.01,1,43.679563,-79.377529
13,"Garden District, Ryerson",0.01,1,43.657162,-79.378937
25,"Richmond, Adelaide, King",0.01,1,43.650571,-79.384568
24,"Regent Park, Harbourfront",0.01,1,43.654260,-79.360636
7,"Commerce Court, Victoria Hotel",0.01,1,43.648198,-79.379817


####Cluster 2

In [58]:
to_clustered.loc[to_clustered['Cluster Labels'] == 2]


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
23,"Queen's Park, Ontario Provincial Government",0.02,2,43.662301,-79.389494
6,Church and Wellesley,0.02,2,43.665860,-79.383160
38,"University of Toronto, Harbord",0.02,2,43.662696,-79.400049


####Observations and Insights
-------
It would be a great idea to open a shopping mall in the city of Toronto as the number of shopping malls present in the city is clearly lacking(i.e. , only 16 in a city of 29.3 lakh people). Most of the shopping malls are concentrated in the central area of Toronto city, with the highest number in cluster 2 and moderate number in cluster 1. On the other hand, cluster 0 has no shopping malls in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is very little to no competition from existing malls. Meanwhile, shopping malls in cluster 2 are likely suffering from intense competition due to the high concentration. 
Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with little to no competition. Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in cluster 1 with moderate competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2.
